In [2]:
import os

from dotenv import load_dotenv

from langchain_community.agent_toolkits.load_tools import load_tools
from langchain_community.tools import AIPluginTool
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.tools.tavily_search.tool import TavilySearchResults

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.tools.retriever import create_retriever_tool

from langchain.agents import AgentExecutor, create_openai_tools_agent, create_react_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI

from langchain import hub

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END

In [3]:
# Load environment variables from .env file
load_dotenv()

# Access the environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
LANGCHAIN_TRACING_V2 = os.getenv('LANGCHAIN_TRACING_V2')
LANGCHAIN_ENDPOINT = os.getenv('LANGCHAIN_ENDPOINT')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
LANGCHAIN_PROJECT = os.getenv('LANGCHAIN_PROJECT')
LANGCHAIN_WANDB_TRACING = os.getenv('LANGCHAIN_WANDB_TRACING')
WANDB_PROJECT = os.getenv('WANDB_PROJECT')
WANDB_API_KEY = os.getenv('WANDB_API_KEY')

# Create tools

In [4]:
### Tools

## Solanalabs API

URL = "https://blockchatstatic.blob.core.windows.net/api-configuration"
solanalabs_tool = load_tools(["requests_post"], allow_dangerous_tools=True)
plugin_tool = AIPluginTool.from_plugin_url(URL + "/.well-known/ai-plugin.json")
solanalabs_tool += [plugin_tool]

## Websearch
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

### Memory

## Long term memory

embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

urls = [
    "https://docs.kamino.finance/",
    "https://docs.kamino.finance/kamino-lend-litepaper",
    "https://docs.kamino.finance/products/overview",
    "https://docs.kamino.finance/products/multiply",
    "https://docs.kamino.finance/products/multiply/how-to",
    "https://docs.kamino.finance/products/multiply/how-to/open-a-position",
    "https://docs.kamino.finance/products/multiply/how-to/manage-a-position",
    "https://docs.kamino.finance/products/multiply/how-to/manage-risk",
    "https://docs.kamino.finance/products/multiply/how-it-works",
    "https://docs.kamino.finance/products/multiply/risks",
    "https://docs.kamino.finance/products/borrow-lend",
    "https://docs.kamino.finance/products/borrow-lend/supplying-assets",
    "https://docs.kamino.finance/products/borrow-lend/ktoken-collateral",
    "https://docs.kamino.finance/products/borrow-lend/borrowing-assets",
    "https://docs.kamino.finance/products/borrow-lend/position-risk-and-liquidations",
    "https://docs.kamino.finance/products/borrow-lend/position-risk-and-liquidations/position-risk",
    "https://docs.kamino.finance/products/borrow-lend/position-risk-and-liquidations/borrow-factors",
    "https://docs.kamino.finance/products/borrow-lend/fees",
    "https://docs.kamino.finance/products/long-short",
    "https://docs.kamino.finance/products/liquidity",
    "https://docs.kamino.finance/kamino-points/overview",
    "https://docs.kamino.finance/kamino-points/overview/rates-and-boosts",
    "https://docs.kamino.finance/kamino-points/overview/seasons",
    "https://docs.kamino.finance/kamino-points/overview/seasons/season-1",
    "https://docs.kamino.finance/kamino-points/overview/seasons/season-2"
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)
vectorstore = Chroma.from_documents(documents=doc_splits,
                                    embedding=embed_model,
                                    collection_name="local-rag")
retriever = vectorstore.as_retriever(search_kwargs={"k":2})

retriever_tool = create_retriever_tool(
    retriever,
    "whitepaper_search",
    "Search for information about crypto project whitepapers. Available whitepapers now are Kamino Finance. For any questions about crypto projects, you must use this tool!",
)

c:\Users\Joshua\AppData\Local\pypoetry\Cache\virtualenvs\custom-openai-api-endpoint-w1RmsYNt-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]


# Helper Utilities

In [5]:
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = hub.pull(system_prompt)
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True)
    return executor

In [6]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

# Create Agent Supervisor

In [7]:
members = ["Blockchain-Researcher", "Internet-Researcher", "Whitepaper-Researcher"]
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
supervisor_prompt = hub.pull("supervisor-prompt").partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI()

supervisor_chain = (
    supervisor_prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

# Construct Graph

In [26]:
# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


blockchain_agent = create_agent(llm, solanalabs_tool, "blockchain-researcher-prompt")
blockchain_node = functools.partial(agent_node, agent=blockchain_agent, name="Blockchain-Researcher")

internet_agent = create_agent(llm, [tavily_tool], "internet-researcher-prompt")
internet_node = functools.partial(agent_node, agent=blockchain_agent, name="Internet-Researcher")

whitepaper_agent = create_agent(llm, [retriever_tool], "whitepaper-researcher-prompt")
whitepaper_node = functools.partial(agent_node, agent=blockchain_agent, name="Whitepaper-Researcher")

workflow = StateGraph(AgentState)
workflow.add_node("Blockchain-Researcher", blockchain_node)
workflow.add_node("Internet-Researcher", internet_node)
workflow.add_node("Whitepaper-Researcher", whitepaper_node)
workflow.add_node("supervisor", supervisor_chain)

In [27]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()

# Invoke the team

In [28]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="How many lamports does 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625 have?")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Blockchain-Researcher'}}
----


> Entering new AgentExecutor chain...

Invoking: `solana` with `{}`


Usage Guide: This extension is for exploring Solana blockchain data, such as inspecting what tokens a wallet has or explaining what happened in a transaction. Use it whenever a user asks something that might be related to their Solana account or transaction history.

OpenAPI Spec: {'openapi': '3.0.2', 'info': {'title': 'Solana Labs API', 'description': 'An API for retrieving human-readable information about the Solana blockchain.', 'version': '1.0.0'}, 'servers': [{'url': 'https://chatgpt.solanalabs.com/'}], 'paths': {'/api/handlers/getAssetsByOwner': {'post': {'summary': 'getAssetsByOwner', 'description': 'Accepts Solana publicKey address. Returns Metaplex NFTs owned by the address', 'operationId': 'query_assets_by_owner', 'requestBody': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/getAccountInfoRequest'}}}, 'required': True}, 're

TypeError: RequestsPostTool._run() got an unexpected keyword argument 'url'